<a href="https://colab.research.google.com/github/ramory-l/DataScience-misis/blob/main/14g_practicum19_09_first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Group 14**

## **Условие Задачи**





Мы с Тамарой (парные простые)
Специалистка по нейронным сетям Бакаева любит искать простые числа. Сначала она решила найти простые числа идущие подряд. Очень быстро она обнаружила, что существует только одна пара таких чисел: (2 3).
Тогда она решила поискать простые числа, идущие через одно число. Например, пара (5 7) удовлетворяет такому условию. Оказалось, что таких чисел много.
Помогите г-же Бакаевой найти количество пар и сумму первых элементов пар простых чисел, оба из которых лежат в промежутке от a до b включительно и между ними расположено ровно одно число (т. е., b - a == 2).

Ограничения: 2 <= a < b <= 10 ** 7.

```
Пример 1.
a = 2, b = 20
Ответ: 4, 36
Пояснение. Вот все парные простые числа от 2 до 20:
3 5
5 7
11 13
17 19
Всего 4 пары, а сумма 3+5+11+17=36
```

```
Пример 2.
a = 203, b = 10000000
Ответ: 58965 272839238019
```

## **Решение**

In [ ]:
def get_prime_grid(n):
    grid = [True] * n
    grid[2::2] = [False] * ((n - 3) // 2 + 1)
    
    for i in range(3, int(n ** 0.5) + 1, 2):
        if grid[i]:
            grid[i ** 2::i * 2] = [False] * ((n - 1 - i ** 2) // (i * 2) + 1)
    
    return grid


def get_primes_pairs_count(a, b):
    pairs_count = 0
    sum_first = 0
    
    primes_grid = get_prime_grid(b)
    
    start = a + 1 if a % 2 == 0 else a
    prev_prime = start
    
    for i in range(start, b, 2):
        if primes_grid[i]:
            if i - prev_prime == 2:
                sum_first += prev_prime
                pairs_count += 1
            
            prev_prime = i
    
    return pairs_count, sum_first

In [ ]:
%%time 
get_primes_pairs_count(203, 10 ** 7)

CPU times: user 590 ms, sys: 6.48 ms, total: 596 ms
Wall time: 599 ms


(58965, 272839238019)

## **Лев**

Самый простой способ нахождения простого числа.

In [ ]:
def is_prime(n):
    return n > 1 and all(n % d for d in range(2, int(n ** 0.5) + 1))

Этот же способ, но используя итератор. Он немного быстрее.

In [ ]:
from itertools import count, islice

def is_prime(n):
    return n > 1 and all(n % i for i in islice(count(2), int(n ** 0.5 - 1)))

**Тест Миллера-Рабина**

Для ультрабольших чисел. Для нашей задачи не подойдет - долго.

In [ ]:
import random

def even_odd(n):
    s, d = 0, n
    while d % 2 == 0:
          s += 1
          d >>= 1
    return s, d

def Miller_Rabin(a, p):
    s, d = even_odd(p - 1)
    a = pow(a, d, p)
    if a == 1: return True
    for i in range(s):
        if a == p - 1: return True
        a = pow(a, 2, p)
    return False

def is_prime(p):
    if p == 2: return True
    if p <= 1 or p % 2 == 0: return False
    return all(Miller_Rabin(random.randint(2,p - 1),p) for _ in range(40))

Все они вызываются следующей функцией, которая дает ответ на вопрос задачи:

In [ ]:
def sum_first_prime(a,b):
    sum = 0
    count = 0
    for i in range(a, b - 1):
        if is_prime(i) and is_prime(i + 2):
            count += 1
            sum += i
    return count, sum

**Решето Эратосфена**

Улучшение реализации решета Эратосфена от Дмитрия декоратором "@njit".

Суть этого декоратора в том, что он эффективно переводит питоновский код в машинный, что позволяет сильно ускорить этот алгоритм.

In [ ]:
from numba import njit

@njit
def get_primes_grid(end):
    lst = [i == 2 or i % 2 != 0 for i in range(end + 1)]
    
    prime = 3
    while prime <= end ** 0.5:
        for j in range(prime ** 2, end + 1, prime):
            lst[j] = False
        
        prime += 2
    
    return lst

@njit
def get_primes_pairs_count(a, b):
    pairs_count = 0
    sum_first = 0
    
    primes_grid = get_primes_grid(b)
    
    start = a + 1 if a % 2 == 0 else a
    prev_prime = start
    
    for i in range(start, b, 2):
        if primes_grid[i]:
            if i - prev_prime == 2:
                sum_first += prev_prime
                pairs_count += 1

            prev_prime = i
    
    return pairs_count, sum_first

In [ ]:
%%time
get_primes_pairs_count(203, 10 ** 7)

CPU times: user 863 ms, sys: 33.2 ms, total: 896 ms
Wall time: 963 ms


(58965, 272839238019)

## **Михаил**

***Алгоритм - Решето Эратосфена***

In [ ]:
# Находим простые числа между 2 и max_number (включительно)
def prime_pairs_inrange(a, max_number):
    # Определяем массив для чисел.
    is_composite = [False for i in range(max_number + 1)]

    # Исключаем числа,кратные 2.
    for i in range(4, max_number + 1, 2):
        is_composite[i] = True

    # Исключаем числа, кратные найденным простым числам.
    next_prime = 3
    stop_at = max_number ** 0.5
    while next_prime <= stop_at:
        # Исключаем числа, кратные данному простому числу.
        for i in range(next_prime * 2, max_number + 1, next_prime):
            is_composite[i] = True
        # Переходим к следующему простому числу, пропуская четные числа.
        next_prime = next_prime + 2
        while next_prime <= max_number and is_composite[next_prime]:
            next_prime += 2

    # Находим пары, такой же алгоритм, как у Дмитрия
    result = 0
    pairs = 0

    start = a + 1 if a % 2 == 0 else a
    pair_start = start

    for i in range(start + 2, max_number + 1, 2):
        if not is_composite[i]:
            if i - pair_start == 2:
                result += pair_start
                pairs += 1
            pair_start = i

    return pairs, result

In [ ]:
%%time
prime_pairs_inrange(2, 20)

CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 16 µs


(4, 36)

In [ ]:
%%time
prime_pairs_inrange(203, 1000000)

CPU times: user 235 ms, sys: 5.77 ms, total: 240 ms
Wall time: 242 ms


(8154, 3712173858)

In [ ]:
%%time
prime_pairs_inrange(203, 10 ** 7)

CPU times: user 2.57 s, sys: 37.5 ms, total: 2.61 s
Wall time: 2.62 s


(58965, 272839238019)

## **Дмитрий**


In [ ]:
def is_prime(num):
    for i in range(2, int(num ** 0.5) + 1):
        if num % i == 0:
            return False
    
    return True


def get_prime_count(a, b):
    res = 0
    pairs = 0
    
    start = a + 1 if a % 2 == 0 else a
    pair_start = start
    
    for i in range(start + 2, b + 1, 2):
        if is_prime(i):
            if i - pair_start == 2:
                res += pair_start
                pairs += 1
            pair_start = i
    
    return pairs, res

In [ ]:
%%time
get_prime_count(2, 20)

CPU times: user 57 µs, sys: 2 µs, total: 59 µs
Wall time: 61.3 µs


(4, 36)

In [ ]:
%%time
get_prime_count(203, 1000000)

CPU times: user 4.24 s, sys: 0 ns, total: 4.24 s
Wall time: 4.25 s


(8154, 3712173858)

In [ ]:
%%time
get_prime_count(203, 10 ** 7)

CPU times: user 1min 59s, sys: 0 ns, total: 1min 59s
Wall time: 1min 59s


(58965, 272839238019)

**Алгоритм - Решето Эратосфена**


In [ ]:
def get_primes_grid(end):
    lst = [True if i == 2 or i % 2 != 0 else False for i in range(end + 1)]
    
    prime = 3
    while prime <= end ** 0.5:
        for j in range(prime ** 2, end + 1, prime):
            lst[j] = False
        
        prime += 2
        while not lst[prime]:
            prime += 2
    
    return lst


def get_primes_pairs_count(a, b):
    pairs_count = 0
    sum_first = 0
    
    primes_grid = get_primes_grid(b)
    
    start = a + 1 if a % 2 == 0 else a
    prev_prime = start
    
    for i in range(start, b, 2):
        if primes_grid[i]:
            if i - prev_prime == 2:
                sum_first += prev_prime
                pairs_count += 1
            
            prev_prime = i
    
    return pairs_count, sum_first

In [ ]:
%%time
get_primes_pairs_count(203, 10 ** 7)

CPU times: user 2.81 s, sys: 152 ms, total: 2.96 s
Wall time: 2.98 s


(58965, 272839238019)

## **Наталья**

In [ ]:
# Функция, определяющая простое число или нет в интервале от 2 до n
def is_prime (n):
    for i in range (2, int(n**0.5)):
        if n % i == 0:
            return False
    return True

# Функция поиска количества пар простых чисел и  сумму первых элементов пар простых чисел, 
# оба из которых лежат в промежутке от a до b включительно и между ними расположено ровно одно число

def pairs_and_sum_first (a, b):
    pairs = 0
    sum_of_first = 0
    for i in range (a, b):
        if is_prime (i)  and is_prime(i + 2):
            pairs += 1
            sum_of_first += i
    return pairs, sum_of_first

In [ ]:
%%time
pairs_and_sum_first(203, 10 ** 7)

CPU times: user 2min 22s, sys: 17 ms, total: 2min 22s
Wall time: 2min 22s


(59085, 273136134039)

# **Саид**

In [ ]:
%%time
#==============================================
#v1 - первое что в голову пришло, решение простой перебор ~20мин
#==============================================
def is_prime(n):
  for i in range(2, n+1):
    if n%i == 0 and i != 1 and i != n:
      return False
  return True
def get_prime_numbers_1(start, end):
  stack = []
  for i in range(start, end, 2):
    if is_prime(i):
      stack.append(i)
  return stack
#==============================================
#v2 - чуть прикинул так ~20мин
#==============================================
max_num = 1000000
nums = list(range(1,max_num,2))
del nums[0]
def exclude_non_prime(num, v1):
  for v in nums[num*num::]:
    if (v%v1 == 0 and v != v1):
      nums.remove(v)
def get_prime_numbers_2(n):
  max_num = n
  for num, value in enumerate(nums):
    exclude_non_prime(num, value)
  return nums
#print(len(nums))
#print(nums)

#==============================================
#v3 - потом так ~3,5мин
#==============================================
x1 = 1
x3 = 3
x7 = 7
x9 = 9
max = 10000000
d = list(range(1,max-1,2))
stack = []

def get_primes(n):
  s1 = True
  s3 = True
  s7 = True
  s9 = True
  
  for i in d[1:int(n**0.5)-1:]:
    if (s1 and (n+x1)%i == 0):
      s1 = False
    if (s3 and (n+x3)%i == 0):
      s3 = False
    if (s7 and (n+x7)%i == 0):
      s7 = False
    if (s9 and (n+x9)%i == 0):
      s9 = False
  if (s1):
    stack.append(n+x1)
  if (s3):
    stack.append(n+x3)
  if (s7):
    stack.append(n+x7)
  if (s9):
    stack.append(n+x9)

def get_prime_numbers_3(n):
  max = n
  for i in range(10, max, 10):
    get_primes(i)
  return stack

#==============================================
#v4 - а потом начал гуглить ~1,2сек
#==============================================

def get_prime_numbers_4( n):
    sieve = [True] * n
    for i in range(3,int(n**0.5)+1,2):
        if sieve[i]:
            sieve[i*i::2*i]=[False]*((n-i*i-1)//(2*i)+1)
    return [2] + [i for i in range(3,n,2) if sieve[i]]


#==============================================
#Решение: используем метод v4
#==============================================

def create_paired_numbers_by_delta(prime_numbers, start, delta):
  stack = []
  max = len(prime_numbers)
  for index, current_number in enumerate(prime_numbers):
    if current_number <= start:
      continue
    if index+1 == max:
      break
    next_number = prime_numbers[index+1]
    if current_number+delta == next_number:
      stack.append([current_number, next_number])
  return stack
def calc_paired_numbers_by_index(array, index):
  sum = 0
  for i in array:
    sum += i[index]
  return sum

#==============================================
prime_numbers = get_prime_numbers_4(10000000)
paired_array = create_paired_numbers_by_delta(prime_numbers, 203,2)
#print(paired_array)
print(len(paired_array))
print(calc_paired_numbers_by_index(paired_array, 0))

58965
272839238019
CPU times: user 945 ms, sys: 106 ms, total: 1.05 s
Wall time: 1.05 s
